#Первый этап обучения AlphaGo.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
#забираем из Google Drive если есть)))
!cp ./drive/MyDrive/colab1/dlgo.zip ./dlgo.zip

In [2]:
#распаковка модулей DL-Go
!unzip -o dlgo.zip

Archive:  dlgo.zip
   creating: dlgo/
   creating: dlgo/agent/
  inflating: dlgo/agent/base.py      
  inflating: dlgo/agent/helpers.py   
  inflating: dlgo/agent/naive.py     
  inflating: dlgo/agent/pg.py        
  inflating: dlgo/agent/predict.py   
  inflating: dlgo/agent/termination.py  
 extracting: dlgo/agent/__init__.py  
   creating: dlgo/agent/__pycache__/
  inflating: dlgo/agent/__pycache__/base.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/base.cpython-36.pyc  
  inflating: dlgo/agent/__pycache__/helpers.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/helpers.cpython-36.pyc  
  inflating: dlgo/agent/__pycache__/naive.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/naive.cpython-36.pyc  
  inflating: dlgo/agent/__pycache__/predict.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/predict.cpython-36.pyc  
  inflating: dlgo/agent/__pycache__/termination.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/termination.cpython-36.pyc  
  inflating: dlgo/

##Загрузка данных для первого этапа обучения сети политики AlphaGo

In [3]:
from dlgo.data.parallel_processor import GoDataProcessor
from dlgo.encoders.alphago import AlphaGoEncoder
from dlgo.agent.predict import DeepLearningAgent
from dlgo.networks.alphago import alphago_model

from keras.callbacks import ModelCheckpoint
import h5py

In [6]:
rows, cols = 19, 19
num_classes = rows * cols
#num_games = 10000 #---> memory error
num_games = 1000 #---> no memory error / memory usage -- 7.5GB
#num_games = 3000 #---> no memory error / memory usage -- 15.7GB

In [7]:
encoder = AlphaGoEncoder()
processor = GoDataProcessor(encoder=encoder.name())
generator = processor.load_go_data('train', num_games, use_generator=True)
test_generator = processor.load_go_data('test', num_games, use_generator=True)

>>> Чтение кэшированной индексной страницы
KGS-2019_04-19-1255-.tar.gz 1255
KGS-2019_03-19-1478-.tar.gz 1478
KGS-2019_02-19-1412-.tar.gz 1412
KGS-2019_01-19-2095-.tar.gz 2095
KGS-2018_12-19-1992-.tar.gz 1992
KGS-2018_11-19-1879-.tar.gz 1879
KGS-2018_10-19-1209-.tar.gz 1209
KGS-2018_09-19-1587-.tar.gz 1587
KGS-2018_08-19-1447-.tar.gz 1447
KGS-2018_07-19-949-.tar.gz 949
KGS-2018_06-19-1002-.tar.gz 1002
KGS-2018_05-19-1590-.tar.gz 1590
KGS-2018_04-19-1612-.tar.gz 1612
KGS-2018_03-19-833-.tar.gz 833
KGS-2018_02-19-1167-.tar.gz 1167
KGS-2018_01-19-1526-.tar.gz 1526
KGS-2017_12-19-1488-.tar.gz 1488
KGS-2017_11-19-945-.tar.gz 945
KGS-2017_10-19-1351-.tar.gz 1351
KGS-2017_09-19-1353-.tar.gz 1353
KGS-2017_08-19-2205-.tar.gz 2205
KGS-2017_07-19-1191-.tar.gz 1191
KGS-2017_06-19-910-.tar.gz 910
KGS-2017_05-19-847-.tar.gz 847
KGS-2017_04-19-913-.tar.gz 913
KGS-2017_03-19-717-.tar.gz 717
KGS-2017_02-19-525-.tar.gz 525
KGS-2017_01-19-733-.tar.gz 733
KGS-2016_12-19-1208-.tar.gz 1208
KGS-2016_11-19-980

##Создание сети политики AlphaGo с помощью Keras

In [8]:
input_shape = (encoder.num_planes, rows, cols)
alphago_sl_policy = alphago_model(input_shape, is_policy_net=True)

In [9]:
alphago_sl_policy.compile('sgd', 'categorical_crossentropy', metrics=['accuracy'])

##Обучение и сохранение сети политики

In [11]:
epochs = 10#200
batch_size = 128
alphago_sl_policy.fit_generator(
    generator=generator.generate(batch_size, num_classes),
    epochs=epochs,
    steps_per_epoch=generator.get_num_samples()/batch_size,
    validation_data=test_generator.generate(batch_size, num_classes),
    validation_steps=test_generator.get_num_samples()/batch_size,
    callbacks=[ModelCheckpoint('alphago_sl_policy_{epoch}.h5')]
)

<ipython-input-11-b4f3c849b3b7>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  alphago_sl_policy.fit_generator(


Epoch 1/10
1480/1480 [==============================] - 429s 290ms/step - loss: 3.6891 - accuracy: 0.4291 - val_loss: 3.6455 - val_accuracy: 0.4380
Epoch 2/10
1480/1480 [==============================] - 472s 319ms/step - loss: 3.6289 - accuracy: 0.4337 - val_loss: 3.5991 - val_accuracy: 0.4386
Epoch 3/10
1480/1480 [==============================] - 473s 319ms/step - loss: 3.5833 - accuracy: 0.4370 - val_loss: 3.5655 - val_accuracy: 0.4431
Epoch 4/10
1480/1480 [==============================] - 473s 320ms/step - loss: 3.5449 - accuracy: 0.4400 - val_loss: 3.5393 - val_accuracy: 0.4488
Epoch 5/10
1480/1480 [==============================] - 428s 290ms/step - loss: 3.5084 - accuracy: 0.4424 - val_loss: 3.5174 - val_accuracy: 0.4512
Epoch 6/10
1480/1480 [==============================] - 427s 289ms/step - loss: 3.4734 - accuracy: 0.4443 - val_loss: 3.5035 - val_accuracy: 0.4484
Epoch 7/10
1480/1480 [==============================] - 470s 318ms/step - loss: 3.4403 - accuracy: 0.4466 - val_

In [12]:
alphago_sl_agent = DeepLearningAgent(alphago_sl_policy, encoder)

In [13]:
with h5py.File('alphago_sl_policy.h5', 'w') as sl_agent_out:
    alphago_sl_agent.serialize(sl_agent_out)

In [14]:
!cp ./alphago_sl_policy.h5 ./drive/MyDrive/colab1/alphago_sl_policy.h5